# Determine derivatives of angular sequence to rotation matrix

Peter Corke 2021

SymPy code to evaluate the mappings from rotation angle sequences to rotation matrix, then compute the derivatives to determine the mapping from angular rates to angular velocity.

Adjust the next cell to choose the particular angle sequence.

In [17]:
from spatialmath.base import *
from sympy import *

In [18]:
# func = eul2r
# angle_names =  ('phi', 'theta', 'psi')

func = lambda Gamma: rpy2r(Gamma, order='yxz')
angle_names = ('alpha', 'beta', 'gamma')


Define a symbol for time

In [19]:
t = symbols('t')

Define arrays of symbols for the angles, angle as function of time, time derivative of angle as a function of time

In [20]:
angle = []  # names of angles, eg. theta
anglet = []  # angles as function of time, eg. theta(t)
angled = []  # derivative of above, eg. d theta(t) / dt
angledn = []  # symbol to represent above, eg. theta_dot
for i in angle_names:
    angle.append(symbols(i))
    anglet.append(Function(i)(t))
    angled.append(anglet[-1].diff(t))
    angledn.append(i + '_dot')

Compute the rotation matrix

In [21]:
R = Matrix(func(anglet))
R

Matrix([
[sin(alpha(t))*sin(beta(t))*sin(gamma(t)) + cos(alpha(t))*cos(gamma(t)), -sin(alpha(t))*cos(gamma(t)) + sin(beta(t))*sin(gamma(t))*cos(alpha(t)), sin(gamma(t))*cos(beta(t))],
[                                            sin(alpha(t))*cos(beta(t)),                                              cos(alpha(t))*cos(beta(t)),              -sin(beta(t))],
[sin(alpha(t))*sin(beta(t))*cos(gamma(t)) - sin(gamma(t))*cos(alpha(t)),  sin(alpha(t))*sin(gamma(t)) + sin(beta(t))*cos(alpha(t))*cos(gamma(t)), cos(beta(t))*cos(gamma(t))]])

Compute its time derivative

In [22]:
Rdot = Matrix(R).diff(t)
Rdot

Matrix([
[ sin(alpha(t))*sin(beta(t))*cos(gamma(t))*Derivative(gamma(t), t) + sin(alpha(t))*sin(gamma(t))*cos(beta(t))*Derivative(beta(t), t) - sin(alpha(t))*cos(gamma(t))*Derivative(alpha(t), t) + sin(beta(t))*sin(gamma(t))*cos(alpha(t))*Derivative(alpha(t), t) - sin(gamma(t))*cos(alpha(t))*Derivative(gamma(t), t), -sin(alpha(t))*sin(beta(t))*sin(gamma(t))*Derivative(alpha(t), t) + sin(alpha(t))*sin(gamma(t))*Derivative(gamma(t), t) + sin(beta(t))*cos(alpha(t))*cos(gamma(t))*Derivative(gamma(t), t) + sin(gamma(t))*cos(alpha(t))*cos(beta(t))*Derivative(beta(t), t) - cos(alpha(t))*cos(gamma(t))*Derivative(alpha(t), t), -sin(beta(t))*sin(gamma(t))*Derivative(beta(t), t) + cos(beta(t))*cos(gamma(t))*Derivative(gamma(t), t)],
[                                                                                                                                                                                                           -sin(alpha(t))*sin(beta(t))*Derivative(beta(t), t) + cos(alpha(t

Get angular velocity vector in terms of angles and angle rates

In [23]:
omega = Matrix(vex(Rdot * R.T))
omega

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               -(sin(alpha(t))*sin(gamma(t)) + sin(beta(t))*cos(alpha(t))*cos(gamma(t)))*(-sin(alpha(t))*cos(beta(t))*Derivative(alpha(t), t) - sin(beta(t))*cos(alpha(t))*Derivative(beta(t), t))/2 - (sin(alpha(t))*sin(beta(t))*cos(gamma(t)) - sin(gamma(t))*cos(alpha(t)))*(-sin(alpha(t))*sin(beta(t))*Derivative(beta(t), t) + cos(alpha(t))*cos(beta(t))*Derivative(alpha(t), t))/2 - (-sin(beta(t))*cos(gamma(t))*Der

For each element of this 3x1 matrix get the coefficients of each angle derivative

In [24]:
A = Matrix.zeros(3,3)
for i in range(3):
    e = omega[i,0].expand()
    for j in range(3):
        A[i, j] = e.coeff(angled[j])

The result is a 3x3 matrix. Mapping from angle rates to angular velocity. We subsitute angle as a function of time to plain angle, then simplify.

In [25]:
A = trigsimp(A.subs(a for a in zip(anglet, angle)))
A

Matrix([
[sin(gamma)*cos(beta),  cos(gamma), 0],
[          -sin(beta),           0, 1],
[cos(beta)*cos(gamma), -sin(gamma), 0]])

Compute the inverse and simplify.  Mapping from angular velocity to angle rates.

In [26]:
Ai = trigsimp(A.inv())
Ai

Matrix([
[sin(gamma)/cos(beta), 0, cos(gamma)/cos(beta)],
[          cos(gamma), 0,          -sin(gamma)],
[sin(gamma)*tan(beta), 1, cos(gamma)*tan(beta)]])

Render as code

In [27]:
pycode(A).replace('ImmutableDenseMatrix', 'np.array')

'np.array([[math.sin(gamma)*math.cos(beta), math.cos(gamma), 0], [-math.sin(beta), 0, 1], [math.cos(beta)*math.cos(gamma), -math.sin(gamma), 0]])'

In [28]:
pycode(Ai).replace('ImmutableDenseMatrix', 'np.array')

'np.array([[math.sin(gamma)/math.cos(beta), 0, math.cos(gamma)/math.cos(beta)], [math.cos(gamma), 0, -math.sin(gamma)], [math.sin(gamma)*math.tan(beta), 1, math.cos(gamma)*math.tan(beta)]])'

Compute the time derivative of `Ai`, from angular acceleration to angle acceleration

In [29]:
Ai = Ai.subs(a for a in zip(angle, anglet))
Ai

Matrix([
[sin(gamma(t))/cos(beta(t)), 0, cos(gamma(t))/cos(beta(t))],
[             cos(gamma(t)), 0,             -sin(gamma(t))],
[sin(gamma(t))*tan(beta(t)), 1, cos(gamma(t))*tan(beta(t))]])

In [30]:
Ai_dot = trigsimp(Ai.diff(t).subs(a for a in zip(angled, angledn)).subs(a for a in zip(anglet, angle)))
Ai_dot

Matrix([
[(beta_dot*sin(beta)*sin(gamma)/cos(beta) + gamma_dot*cos(gamma))/cos(beta), 0, (beta_dot*sin(beta)*cos(gamma)/cos(beta) - gamma_dot*sin(gamma))/cos(beta)],
[                                                     -gamma_dot*sin(gamma), 0,                                                      -gamma_dot*cos(gamma)],
[         beta_dot*sin(gamma)/cos(beta)**2 + gamma_dot*cos(gamma)*tan(beta), 0,          beta_dot*cos(gamma)/cos(beta)**2 - gamma_dot*sin(gamma)*tan(beta)]])

In [31]:
pycode(Ai_dot).replace('ImmutableDenseMatrix', 'np.array')

'np.array([[(beta_dot*math.sin(beta)*math.sin(gamma)/math.cos(beta) + gamma_dot*math.cos(gamma))/math.cos(beta), 0, (beta_dot*math.sin(beta)*math.cos(gamma)/math.cos(beta) - gamma_dot*math.sin(gamma))/math.cos(beta)], [-gamma_dot*math.sin(gamma), 0, -gamma_dot*math.cos(gamma)], [beta_dot*math.sin(gamma)/math.cos(beta)**2 + gamma_dot*math.cos(gamma)*math.tan(beta), 0, beta_dot*math.cos(gamma)/math.cos(beta)**2 - gamma_dot*math.sin(gamma)*math.tan(beta)]])'